In [1]:
import pandas as pd
import constant as c
import geopandas as gpd
import numpy as np
from shapely.wkt import loads

In [2]:
df_grids = pd.read_csv('../asset/preprocess/grid/df_grid_pop_od_ebit.csv', index_col = 0) # grid 기준: 각 grid에 pop, od 데이터가 옆으로 추가된 형태
gdf_cbds = pd.read_csv("../asset/preprocess/grid/gdf_cbds.csv", index_col = 0) # building 기준: 각 building에 feature 데이터가 옆으로 추가된 형태
seoul_bdry = pd.read_csv('../asset/preprocess/grid/seoul_boundary.csv', index_col = 0) # bdry -> boundary
scbd_bdry = pd.read_csv('../asset/preprocess/grid/scbd_boundary.csv', index_col = 0) # bdry -> boundary
gbd_bdry = pd.read_csv('../asset/preprocess/grid/gbd_boundary.csv', index_col = 0) # bdry -> boundary
ybd_bdry = pd.read_csv('../asset/preprocess/grid/ybd_boundary.csv', index_col = 0)

In [3]:
gdf_cbds['year'] = gdf_cbds['useaprday'].astype(str).str[:4]
gdf_cbds['year'] = pd.to_numeric(gdf_cbds['year'], errors='coerce')

### 2022

In [4]:
# 2022
YEAR = '2022'
GRID_INT_COLS = ['grid_idx', f'pop{YEAR}', f'od{YEAR}', f'ebit{YEAR}', 'geometry_grids']
COLS_RENAME_DICT = {
    f'pop{YEAR}': 'pp_pop', 
    f'od{YEAR}': 'pp_od', 
    f'ebit{YEAR}': 'bs_ebit'
}
df_grid_2022 = df_grids.copy()
df_grid_2022 = df_grid_2022[GRID_INT_COLS]
df_grid_2022 = df_grid_2022.rename(columns=COLS_RENAME_DICT)

df_bd_2022 = gdf_cbds.copy()
df_bd_2022['totflrcnt'] = df_bd_2022['grndflrcnt'] + df_bd_2022['ugrndflrcnt']
df_bd_2022['vintage'] = int(YEAR) - df_bd_2022['year']

COLS_RENAME_DICT = {
    'platarea': 'bd_platarea',
    'archarea': 'bd_archarea',  
    'totarea': 'bd_totarea', 
    'totflrcnt': 'bd_totflrcnt', 
    'vintage': 'bd_vintage', 
    'rideuseelvtcnt': 'bd_elvtent',
    'heit': 'bd_height',
    f'ilp{YEAR}': 'bd_ilp', 
    f'{YEAR}gas': 'bs_gas', 
    f'{YEAR}elct': 'bs_elct'
}
df_bd_2022 = df_bd_2022.rename(columns=COLS_RENAME_DICT)
BD_INT_COLS = ['grid_idx', 'sigungunm','platplc','x', 'y', 'x_5179', 'y_5179','bdnm', 'year',\
    'bd_platarea', 'bd_archarea', 'bd_totarea','bd_totflrcnt', 'bd_height', 'bd_vintage', 'bd_elvtent', 'bd_ilp','bs_gas', 'bs_elct']
df_bd_2022 = df_bd_2022[BD_INT_COLS]
df_bd_2022_grouped = df_bd_2022.groupby('grid_idx').agg({
    'sigungunm': 'first',
    'platplc': 'first',
    'x': 'first',
    'y': 'first',
    'x_5179': 'first',
    'y_5179': 'first',
    'bdnm': lambda x: ', '.join(str(val) for val in x if not pd.isna(val)) if any(not pd.isna(val) for val in x) else 'NoData',
    'bd_platarea': 'sum',
    'bd_archarea': 'sum',
    'bd_totarea': 'sum',
    'bd_totflrcnt': 'sum',
    'bd_elvtent': 'mean',
    'bs_gas': 'sum',
    'bs_elct': 'sum',
    'bd_height': 'mean',
    'bd_ilp': 'mean',
    'bd_vintage': 'mean'
}).reset_index()

df_bd_2022_grouped['count'] = df_bd_2022.groupby('grid_idx')['grid_idx'].transform('size')
# Perform left join on 'grid_idx'
df_2022 = pd.merge(df_grid_2022, df_bd_2022_grouped, on='grid_idx', how='inner')
df_2022['geometry_grids'] = df_2022['geometry_grids'].apply(lambda x: loads(x))

### 2021

In [5]:
# 2021
YEAR = '2021'
GRID_INT_COLS = ['grid_idx', f'pop{YEAR}', f'od{YEAR}', f'ebit{YEAR}', 'geometry_grids']
COLS_RENAME_DICT = {
    f'pop{YEAR}': 'pp_pop', 
    f'od{YEAR}': 'pp_od', 
    f'ebit{YEAR}': 'bs_ebit'
}
df_grid_2021 = df_grids.copy()
df_grid_2021 = df_grid_2021[GRID_INT_COLS]
df_grid_2021 = df_grid_2021.rename(columns=COLS_RENAME_DICT)

df_bd_2021 = gdf_cbds.copy()
df_bd_2021['totflrcnt'] = df_bd_2021['grndflrcnt'] + df_bd_2021['ugrndflrcnt']
df_bd_2021['vintage'] = int(YEAR) - df_bd_2021['year']
df_bd_2021 = df_bd_2021[df_bd_2021['vintage'] >= 0] # let 2022 building out from df

COLS_RENAME_DICT = {
    'platarea': 'bd_platarea',
    'archarea': 'bd_archarea',  
    'totarea': 'bd_totarea', 
    'totflrcnt': 'bd_totflrcnt', 
    'vintage': 'bd_vintage', 
    'rideuseelvtcnt': 'bd_elvtent',
    'heit': 'bd_height',
    f'ilp{YEAR}': 'bd_ilp', 
    f'{YEAR}gas': 'bs_gas', 
    f'{YEAR}elct': 'bs_elct'
}
df_bd_2021 = df_bd_2021.rename(columns=COLS_RENAME_DICT)
BD_INT_COLS = ['grid_idx', 'sigungunm','platplc','x', 'y', 'x_5179', 'y_5179','bdnm', 'year',\
    'bd_platarea', 'bd_archarea', 'bd_totarea','bd_totflrcnt', 'bd_height', 'bd_vintage', 'bd_elvtent', 'bd_ilp','bs_gas', 'bs_elct']
df_bd_2021 = df_bd_2021[BD_INT_COLS]
df_bd_2021_grouped = df_bd_2021.groupby('grid_idx').agg({
    'sigungunm': 'first',
    'platplc': 'first',
    'x': 'first',
    'y': 'first',
    'x_5179': 'first',
    'y_5179': 'first',
    'bdnm': lambda x: ', '.join(str(val) for val in x if not pd.isna(val)) if any(not pd.isna(val) for val in x) else 'NoData',
    'bd_platarea': 'sum',
    'bd_archarea': 'sum',
    'bd_totarea': 'sum',
    'bd_totflrcnt': 'sum',
    'bd_elvtent': 'mean',
    'bs_gas': 'sum',
    'bs_elct': 'sum',
    'bd_height': 'mean',
    'bd_ilp': 'mean',
    'bd_vintage': 'mean'
}).reset_index()

df_bd_2021_grouped['count'] = df_bd_2021.groupby('grid_idx')['grid_idx'].transform('size')
# Perform left join on 'grid_idx'
df_2021 = pd.merge(df_grid_2021, df_bd_2021_grouped, on='grid_idx', how='inner')
df_2021['geometry_grids'] = df_2021['geometry_grids'].apply(lambda x: loads(x))

### 2020

In [6]:
# 2020
YEAR = '2020'
GRID_INT_COLS = ['grid_idx', f'pop{YEAR}', f'od{YEAR}', f'ebit{YEAR}', 'geometry_grids']
COLS_RENAME_DICT = {
    f'pop{YEAR}': 'pp_pop', 
    f'od{YEAR}': 'pp_od', 
    f'ebit{YEAR}': 'bs_ebit'
}
df_grid_2020 = df_grids.copy()
df_grid_2020 = df_grid_2020[GRID_INT_COLS]
df_grid_2020 = df_grid_2020.rename(columns=COLS_RENAME_DICT)

df_bd_2020 = gdf_cbds.copy()
df_bd_2020['totflrcnt'] = df_bd_2020['grndflrcnt'] + df_bd_2020['ugrndflrcnt']
df_bd_2020['vintage'] = int(YEAR) - df_bd_2020['year']
df_bd_2020 = df_bd_2020[df_bd_2020['vintage'] >= 0] # let 2022, 2021 building out from df

COLS_RENAME_DICT = {
    'platarea': 'bd_platarea',
    'archarea': 'bd_archarea',  
    'totarea': 'bd_totarea', 
    'totflrcnt': 'bd_totflrcnt', 
    'vintage': 'bd_vintage', 
    'rideuseelvtcnt': 'bd_elvtent',
    'heit': 'bd_height',
    f'ilp{YEAR}': 'bd_ilp', 
    f'{YEAR}gas': 'bs_gas', 
    f'{YEAR}elct': 'bs_elct'
}
df_bd_2020 = df_bd_2020.rename(columns=COLS_RENAME_DICT)
BD_INT_COLS = ['grid_idx', 'sigungunm','platplc','x', 'y', 'x_5179', 'y_5179','bdnm', 'year',\
    'bd_platarea', 'bd_archarea', 'bd_totarea','bd_totflrcnt', 'bd_height', 'bd_vintage', 'bd_elvtent', 'bd_ilp','bs_gas', 'bs_elct']
df_bd_2020 = df_bd_2020[BD_INT_COLS]
df_bd_2020_grouped = df_bd_2020.groupby('grid_idx').agg({
    'sigungunm': 'first',
    'platplc': 'first',
    'x': 'first',
    'y': 'first',
    'x_5179': 'first',
    'y_5179': 'first',
    'bdnm': lambda x: ', '.join(str(val) for val in x if not pd.isna(val)) if any(not pd.isna(val) for val in x) else 'NoData',
    'bd_platarea': 'sum',
    'bd_archarea': 'sum',
    'bd_totarea': 'sum',
    'bd_totflrcnt': 'sum',
    'bd_elvtent': 'mean',
    'bs_gas': 'sum',
    'bs_elct': 'sum',
    'bd_height': 'mean',
    'bd_ilp': 'mean',
    'bd_vintage': 'mean'
}).reset_index()

df_bd_2020_grouped['count'] = df_bd_2020.groupby('grid_idx')['grid_idx'].transform('size')
# Perform left join on 'grid_idx'
df_2020 = pd.merge(df_grid_2020, df_bd_2020_grouped, on='grid_idx', how='inner')
df_2020['geometry_grids'] = df_2020['geometry_grids'].apply(lambda x: loads(x))

In [13]:
df_2020.to_csv('../asset/preprocess/df_2020.csv')
df_2021.to_csv('../asset/preprocess/df_2021.csv')
df_2022.to_csv('../asset/preprocess/df_2022.csv')

In [18]:
# then read the data
df_2020 = pd.read_csv('../asset/preprocess/df_2020.csv', index_col=0)
df_2020['geometry_grids'] = df_2020['geometry_grids'].apply(lambda x: loads(x))
df_2021 = pd.read_csv('../asset/preprocess/df_2021.csv', index_col=0)
df_2021['geometry_grids'] = df_2021['geometry_grids'].apply(lambda x: loads(x))
df_2022 = pd.read_csv('../asset/preprocess/df_2022.csv', index_col=0)
df_2022['geometry_grids'] = df_2022['geometry_grids'].apply(lambda x: loads(x))

https://geographicdata.science/book/notebooks/10_clustering_and_regionalization.html